Простой пример: Графические Объекты
Представим редактор, где можно рисовать простые фигуры (круги, квадраты) и группировать их. Мы хотим иметь возможность отрисовать как отдельную фигуру, так и целую группу одинаковым образом.

In [1]:
from abc import ABC, abstractmethod
from typing import List

# 1. Component Interface (Общий интерфейс Компонента)
class Graphic(ABC):
    """Общий интерфейс для всех графических объектов (и простых, и составных)."""
    @abstractmethod
    def draw(self, indent: str = "") -> None:
        """Метод отрисовки."""
        pass

    # Методы для управления детьми (вариант "Uniformity")
    # Листья их просто игнорируют или вызывают ошибку.
    def add(self, component: 'Graphic') -> None:
        raise NotImplementedError("Cannot add to a leaf element")

    def remove(self, component: 'Graphic') -> None:
        raise NotImplementedError("Cannot remove from a leaf element")

    def get_child(self, index: int) -> 'Graphic':
        raise NotImplementedError("Leaf elements have no children")

# 2. Leaf (Лист - простая фигура)
class Circle(Graphic):
    """Простой графический объект - Круг."""
    def __init__(self, name: str):
        self.name = name

    def draw(self, indent: str = "") -> None:
        print(f"{indent}Drawing Circle: {self.name}")

class Square(Graphic):
    """Простой графический объект - Квадрат."""
    def __init__(self, name: str):
        self.name = name

    def draw(self, indent: str = "") -> None:
        print(f"{indent}Drawing Square: {self.name}")

# 3. Composite (Композит - группа фигур)
class GraphicGroup(Graphic):
    """Составной объект - группа графических элементов."""
    def __init__(self, name: str):
        self.name = name
        self._children: List[Graphic] = []

    def add(self, component: Graphic) -> None:
        print(f"Adding {type(component).__name__} to Group '{self.name}'")
        self._children.append(component)

    def remove(self, component: Graphic) -> None:
        print(f"Removing {type(component).__name__} from Group '{self.name}'")
        self._children.remove(component)

    def get_child(self, index: int) -> Graphic:
        return self._children[index]

    def draw(self, indent: str = "") -> None:
        """Отрисовывает группу и рекурсивно отрисовывает всех детей."""
        print(f"{indent}--- Start Group: {self.name} ---")
        for child in self._children:
            child.draw(indent + "  ") # Увеличиваем отступ для детей
        print(f"{indent}--- End Group: {self.name} ---")


# 4. Client Code (Клиентский Код)
if __name__ == "__main__":
    # Создаем простые фигуры (листья)
    circle1 = Circle("C1")
    square1 = Square("S1")
    circle2 = Circle("C2")

    # Создаем группы (композиты)
    group1 = GraphicGroup("Group 1")
    group2 = GraphicGroup("Group 2 (Nested)")

    # Собираем иерархию
    group1.add(circle1)
    group1.add(square1)

    group2.add(circle2) # Добавляем фигуру во вложенную группу

    group1.add(group2) # Добавляем вложенную группу в основную группу

    print("\nDrawing the entire graphic structure (Group 1):")
    # Клиент работает с корневым элементом (group1) через общий интерфейс Graphic
    group1.draw()

    print("\nDrawing just a leaf element (circle1):")
    circle1.draw()

    print("\nDrawing just the nested group (Group 2):")
    group2.draw()

# Вывод:
# Adding Circle to Group 'Group 1'
# Adding Square to Group 'Group 1'
# Adding Circle to Group 'Group 2 (Nested)'
# Adding GraphicGroup to Group 'Group 1'
#
# Drawing the entire graphic structure (Group 1):
# --- Start Group: Group 1 ---
#   Drawing Circle: C1
#   Drawing Square: S1
#   --- Start Group: Group 2 (Nested) ---
#     Drawing Circle: C2
#   --- End Group: Group 2 (Nested) ---
# --- End Group: Group 1 ---
#
# Drawing just a leaf element (circle1):
# Drawing Circle: C1
#
# Drawing just the nested group (Group 2):
# --- Start Group: Group 2 (Nested) ---
#   Drawing Circle: C2
# --- End Group: Group 2 (Nested) ---

Adding Circle to Group 'Group 1'
Adding Square to Group 'Group 1'
Adding Circle to Group 'Group 2 (Nested)'
Adding GraphicGroup to Group 'Group 1'

Drawing the entire graphic structure (Group 1):
--- Start Group: Group 1 ---
  Drawing Circle: C1
  Drawing Square: S1
  --- Start Group: Group 2 (Nested) ---
    Drawing Circle: C2
  --- End Group: Group 2 (Nested) ---
--- End Group: Group 1 ---

Drawing just a leaf element (circle1):
Drawing Circle: C1

Drawing just the nested group (Group 2):
--- Start Group: Group 2 (Nested) ---
  Drawing Circle: C2
--- End Group: Group 2 (Nested) ---


Сложный пример: Файловая Система
Моделируем файловую систему с файлами (листья) и папками (композиты). Хотим иметь возможность рассчитать общий размер как для файла, так и для папки (включая всё её содержимое).

In [2]:
from abc import ABC, abstractmethod
from typing import List, Optional

# 1. Component Interface (Компонент Файловой Системы)
class FileSystemComponent(ABC):
    """Общий интерфейс для файлов и папок."""
    def __init__(self, name: str):
        self._name = name
        self._parent: Optional['Directory'] = None # Ссылка на родителя (опционально)

    @property
    def name(self) -> str:
        return self._name

    @property
    def parent(self) -> Optional['Directory']:
        return self._parent

    @parent.setter
    def parent(self, parent: Optional['Directory']):
        self._parent = parent

    @abstractmethod
    def get_size(self) -> int:
        """Возвращает размер компонента в байтах."""
        pass

    @abstractmethod
    def display(self, indent: str = "") -> None:
        """Отображает структуру компонента."""
        pass

    # Методы управления детьми (не у всех компонентов)
    def add(self, component: 'FileSystemComponent') -> None:
        raise TypeError(f"Cannot add to a {type(self).__name__}")

    def remove(self, component: 'FileSystemComponent') -> None:
        raise TypeError(f"Cannot remove from a {type(self).__name__}")

    def is_composite(self) -> bool:
        return False # По умолчанию - не композит

# 2. Leaf (Лист - Файл)
class File(FileSystemComponent):
    """Представляет файл в файловой системе."""
    def __init__(self, name: str, size: int):
        super().__init__(name)
        self._size = size

    def get_size(self) -> int:
        """Размер файла фиксирован."""
        return self._size

    def display(self, indent: str = "") -> None:
        print(f"{indent}📄 File: {self.name} ({self.get_size()} bytes)")

# 3. Composite (Композит - Папка)
class Directory(FileSystemComponent):
    """Представляет папку (директорию), которая может содержать другие компоненты."""
    def __init__(self, name: str):
        super().__init__(name)
        self._children: List[FileSystemComponent] = []

    def add(self, component: FileSystemComponent) -> None:
        """Добавляет компонент (файл или папку) в эту папку."""
        print(f"Adding '{component.name}' to directory '{self.name}'")
        component.parent = self # Устанавливаем родителя для добавляемого компонента
        self._children.append(component)

    def remove(self, component: FileSystemComponent) -> None:
        """Удаляет компонент из папки."""
        print(f"Removing '{component.name}' from directory '{self.name}'")
        component.parent = None
        self._children.remove(component)

    def is_composite(self) -> bool:
        return True

    def get_size(self) -> int:
        """Размер папки - это сумма размеров всех её дочерних компонентов."""
        total_size = 0
        print(f"Calculating size for directory '{self.name}'...")
        for child in self._children:
            total_size += child.get_size() # Рекурсивный вызов
        return total_size

    def display(self, indent: str = "") -> None:
        """Отображает имя папки и рекурсивно отображает её содержимое."""
        print(f"{indent}📁 Directory: {self.name}")
        for child in self._children:
            child.display(indent + "  ")


# 4. Client Code
if __name__ == "__main__":
    # Создаем структуру файловой системы
    root = Directory("root")
    home = Directory("home")
    user1 = Directory("user1")
    user2 = Directory("user2")

    file1 = File("document.txt", 1024)
    file2 = File("image.jpg", 5120)
    file3 = File("archive.zip", 20480)
    file4 = File("notes.txt", 512)
    config_file = File("config.ini", 256)

    # Собираем иерархию
    root.add(home)
    root.add(config_file) # Файл в корне

    home.add(user1)
    home.add(user2)

    user1.add(file1)
    user1.add(file2)

    user2.add(file3)
    user2.add(file4)

    print("\n--- Displaying File System Structure ---")
    # Клиент работает с корневым каталогом через общий интерфейс
    root.display()

    print("\n--- Calculating Sizes ---")
    print(f"\nSize of file '{file1.name}': {file1.get_size()} bytes")

    print(f"\nCalculating size of directory '{user1.name}':")
    size_user1 = user1.get_size()
    print(f"Total size of '{user1.name}': {size_user1} bytes") # Ожидаем 1024 + 5120 = 6144

    print(f"\nCalculating size of directory '{root.name}':")
    size_root = root.get_size()
    # Ожидаем size_user1 + size_user2 + size_config
    # size_user2 = 20480 + 512 = 20992
    # size_root = 6144 + 20992 + 256 = 27392
    print(f"Total size of '{root.name}': {size_root} bytes")

# Вывод:
# Adding 'home' to directory 'root'
# Adding 'config.ini' to directory 'root'
# Adding 'user1' to directory 'home'
# Adding 'user2' to directory 'home'
# Adding 'document.txt' to directory 'user1'
# Adding 'image.jpg' to directory 'user1'
# Adding 'archive.zip' to directory 'user2'
# Adding 'notes.txt' to directory 'user2'
#
# --- Displaying File System Structure ---
# 📁 Directory: root
#   📁 Directory: home
#     📁 Directory: user1
#       📄 File: document.txt (1024 bytes)
#       📄 File: image.jpg (5120 bytes)
#     📁 Directory: user2
#       📄 File: archive.zip (20480 bytes)
#       📄 File: notes.txt (512 bytes)
#   📄 File: config.ini (256 bytes)
#
# --- Calculating Sizes ---
#
# Size of file 'document.txt': 1024 bytes
#
# Calculating size of directory 'user1':
# Calculating size for directory 'user1'...
# Total size of 'user1': 6144 bytes
#
# Calculating size of directory 'root':
# Calculating size for directory 'root'...
# Calculating size for directory 'home'...
# Calculating size for directory 'user1'...
# Calculating size for directory 'user2'...
# Total size of 'root': 27392 bytes

Adding 'home' to directory 'root'
Adding 'config.ini' to directory 'root'
Adding 'user1' to directory 'home'
Adding 'user2' to directory 'home'
Adding 'document.txt' to directory 'user1'
Adding 'image.jpg' to directory 'user1'
Adding 'archive.zip' to directory 'user2'
Adding 'notes.txt' to directory 'user2'

--- Displaying File System Structure ---
📁 Directory: root
  📁 Directory: home
    📁 Directory: user1
      📄 File: document.txt (1024 bytes)
      📄 File: image.jpg (5120 bytes)
    📁 Directory: user2
      📄 File: archive.zip (20480 bytes)
      📄 File: notes.txt (512 bytes)
  📄 File: config.ini (256 bytes)

--- Calculating Sizes ---

Size of file 'document.txt': 1024 bytes

Calculating size of directory 'user1':
Calculating size for directory 'user1'...
Total size of 'user1': 6144 bytes

Calculating size of directory 'root':
Calculating size for directory 'root'...
Calculating size for directory 'home'...
Calculating size for directory 'user1'...
Calculating size for directory '